# RL for inverted pendulum controlled with muscle

In [1]:
import numpy as np
import mediapy as media
import matplotlib.pyplot as plt

import sys
sys.path.append("../../Biodiffrl")

import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".90"

# Optionally, force JAX to preallocate memory.
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "true"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

# Setup environment variable for Nvidia GPU acceleration
os.environ['XLA_FLAGS'] = (
    "--xla_gpu_triton_gemm_any=true"
    # '--xla_gpu_enable_async_collectives=true '
    # '--xla_gpu_enable_latency_hiding_scheduler=true '
    # '--xla_gpu_enable_highest_priority_async_stream=true '
    # '--xla_cpu_multi_thread_eigen=true intra_op_parallelism_threads=32'
)

backend = 'gpu'
# backend = 'METAL'
# backend = 'cpu'

import jax
jax.config.update('jax_platform_name', 'gpu')
# os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_lazy_compilation=false"
# Enable compliation catch
os.environ["JAX_COMPILATION_CACHE_DIR"] = "./jax_cache"
jax.config.update("jax_compilation_cache_dir", "./jax_cache")
jax.config.update("jax_persistent_cache_min_entry_size_bytes", 0)
jax.config.update("jax_persistent_cache_min_compile_time_secs", 2)
# jax.config.update("jax_explain_cache_misses", True)

# Solve NaN on newer nvidia cards
jax.config.update('jax_default_matmul_precision', "high")

from jax.experimental.compilation_cache import compilation_cache as cc
cc.set_cache_dir("./jax_cache")
# Debug Nan
# jax.config.update("jax_debug_nans", True)

from jax import numpy as jp
# More legible printing from numpy.
jp.set_printoptions(precision=6, suppress=True, linewidth=100)

import mujoco
import mujoco.mjx as mjx
from mujoco.mjx._src import scan
from mujoco.mjx._src import types

# More legible printing from numpy.
np.set_printoptions(precision=6, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

device = jax.devices(backend=backend)[0]

model_path = '/home/bugman/Currentwork/biomujoco_rl/RL/simple_example/inverted_pendulum_mtu.xml'
# model_path = '/home/bugman/Currentwork/biomujoco_converter/converted/mjc/Gait2354/gait2354_cvt1_easy.xml'
# model_path = "/home/bugman/Downloads/Genesis-main/genesis/assets/xml/franka_emika_panda/panda.xml"

# Single step
mjx_step = jax.jit(mjx.step, backend=backend)

In [2]:
from mujoco.mjx._src.biomtu import acceleration_mtu

mj_model = mujoco.MjModel.from_xml_path(model_path)
mjx_model = mjx.put_model(mj_model,device=device)

# Disable tendon
# opt = mjx_model.opt.replace(disableflags = mjx_model.opt.disableflags |mujoco.mjtDisableBit.mjDSBL_PASSIVE)
# mjx_model = mjx_model.replace(opt=opt)

mjx_data = mjx.make_data(mjx_model)
mj_data = mujoco.MjData(mj_model)

# Load the Keyframe
# mjx_data = mjx_data.replace(qpos = mj_model.key_qpos[0])
# mj_data.qpos = mj_model.key_qpos[0]

# Calculate equilibrum
mjx_data = acceleration_mtu.calc_equilibrium(mjx_model, mjx_data)
mjx_data = jax.jit(mjx_step)(mjx_model, mjx_data)

init_mjx_data = mjx_data

def print_all():
    print(mjx_model.biomtu_adr)
    print(mjx_model.mtu_wrap_objid)
    print(mjx_model.mtu_wrap_type)
    print(mjx_model.biomtu_fiso)
    print(mjx_model.biomtu_vmax)
    print(mjx_model.biomtu_ofl)
    print(mjx_model.biomtu_opa)
    print(mjx_model.biomtu_mass)
    print("-------Data--------")
    print("qpos:", mjx_data.qpos)
    print("mtu l:", mjx_data.biomtu.l)
    print("tendon l:", mjx_data.biomtu.tendon_l)
    print("fiber l :", mjx_data.biomtu.fiber_l)
    print("Muscle Bce:", mjx_data.biomtu.B_ce)
    print("Muscle vm:", mjx_data.biomtu.m)
    print("Fiber acc:", mjx_data.biomtu.fiber_acc)
    print("Fiber v:", mjx_data.biomtu.fiber_v)
    print("Biomtu h:", mjx_data.biomtu.h)
    print(mjx_data.biomtu.v)
    print(mjx_data.biomtu.h)  # The constant high of the muscle.
    print(mjx_data.biomtu.pennation_angle)
    print(mjx_data.biomtu.origin_body_id)
    print(mjx_data.biomtu.insertion_body_id)
    print("mtu act:", mjx_data.biomtu.act)
    # print(mjx_data.biomtu.j)
    print(mjx_data.qfrc_biomtu)
    # print(mj_model.key_time)
    # print(mj_model.key_qpos)
    # print(mj_model.key_qvel)

print_all()

# print(mjx_model.nbiomtu)
# print(mjx_model.nq)
# print(mjx_data.qpos)
# print(mjx_data.qvel)

[0 2]
[0 2 1 3]
[3 3 3 3]
[2000. 2000.]
[10. 10.]
[0.9 0.9]
[0.05236 0.05236]
[1. 1.]
-------Data--------
qpos: [-0.  0.]
mtu l: [2.1 2.1]
tendon l: [0.913822 0.913822]
fiber l : [1.187113 1.187113]
Muscle Bce: [0.05 0.05]
Muscle vm: [0.666667 0.666667]
Fiber acc: [0.89215 0.89215]
Fiber v: [0.000474 0.000474]
Biomtu h: [0.047102 0.047102]
[0. 0.]
[0.047102 0.047102]
[0.039689 0.039689]
[0 0]
[1 1]
mtu act: [0. 0.]
[0. 0.]


In [ ]:
import exp

# Multiple steps
def step_fn(carry, _):
    model, data= carry
    new_data = mjx.step(model, data)
    new_carry = (model, new_data)
    return new_carry, _

def multiple_steps(model, data):
    init_carry = (model, data)
    y, _ = jax.lax.scan(step_fn, init_carry, None, length=10)
    new_data = y[1]
    return new_data



jit_multiple_steps = jax.jit(multiple_steps)

In [ ]:
import mujoco.viewer
import time

# Debug Nan
# jax.config.update("jax_debug_nans", True)

mj_model = mujoco.MjModel.from_xml_path(model_path)
mjx_model = mjx.put_model(mj_model,device=device)

# Disable tendon
# opt = mjx_model.opt.replace(disableflags = mjx_model.opt.disableflags |mujoco.mjtDisableBit.mjDSBL_PASSIVE)
# mjx_model = mjx_model.replace(opt=opt)

mjx_data = mjx.make_data(mjx_model)
mj_data = mujoco.MjData(mj_model)

# Calculate equilibrum
mjx_data = acceleration_mtu.calc_equilibrium(mjx_model, mjx_data)

previous_frame_time = time.time()
i = 0
# key = jax.random.key(334)
with mujoco.viewer.launch_passive(mj_model, mj_data) as viewer:
    while viewer.is_running():
        # Update mjx_data from mj_data. The mj_data was modified by the viewer
        # mjx_data = mjx_data.replace(ctrl=mj_data.ctrl, xfrc_applied=mj_data.xfrc_applied)
        # Use the nerual network to generate ctrl signal
        
        mjx_data = mjx_data.replace(xfrc_applied=jp.array(mj_data.xfrc_applied*10, dtype=jp.float32))
        
        # Generate key
        # key = jax.random.split(key,1)[0]
        # xfrc = jax.random.uniform(key,(mjx_model.nbody, 6), minval=-10, maxval=10)
        # mjx_data = mjx_data.replace(xfrc_applied=xfrc)
        mjx_data = mjx_data.replace(
            qpos= jp.array(mj_data.qpos, dtype=jp.float32),
            qvel= jp.array(mj_data.qvel, dtype=jp.float32),
            time = jp.array(mj_data.time, dtype=jp.float32))
        
        # Update mjx_model from mj_model
        mjx_model = mjx_model.tree_replace({
            'opt.gravity': jp.array(mj_model.opt.gravity, dtype=jp.float32),
            'opt.tolerance': jp.array(mj_model.opt.tolerance, dtype=jp.float32),
            'opt.ls_tolerance': jp.array(mj_model.opt.ls_tolerance, dtype=jp.float32),
            'opt.timestep': jp.array(mj_model.opt.timestep, dtype=jp.float32),
        })
        
        
        # Control Muscle
        mjx_data = mjx_data.replace(biomtu = mjx_data.biomtu.replace(act = jp.array([0,0])))
        
        # mjx_data = mjx_step(mjx_model, mjx_data)
        mjx_data = jit_multiple_steps(mjx_model, mjx_data)
        # mjx_data, loss, exps = jit_nn_multi_steps(controller_params, mjx_model, mjx_data, key)
        # mjx_data, key, act = jit_nn_mjx_one_step_no_random(controller_params, mjx_model, mjx_data, key)
        
        mjx.get_data_into(mj_data, mj_model, mjx_data)
        
        # Record the current time at the start of this frame
        current_frame_time = time.time()
    
        # Calculate the difference in time from the last frame
        time_between_frames = current_frame_time - previous_frame_time
    
        # Print the time between frames
        print(f"Time between frames: {time_between_frames} seconds")
        previous_frame_time = current_frame_time
        
        # print("ACT:", mjx_data.biomtu.act)
        print(mjx_data.qpos)
        # print(mj_data.sen)  
        # print(mjx_data.sensordata[3:6])
        # print(mjx_data.biomtu.act)
        # print(mjx_data.qfrc_inverse[6], mjx_data.qfrc_inverse[15] )
        # print(mjx_data.qfrc_constraint[6], mjx_data.qfrc_constraint[15])
        # print(len(mjx_data.qvel))
        
        
        viewer.sync()

## Neural Network

In [3]:
import nn

key = jax.random.key(6848)
# Controller NN
controller_nn = nn.Controller_NN(mjx_model.nq*2+mjx_model.nbiomtu*2, 2)     # There are two muscle so two output
controller_params, key = controller_nn.init_parameters(key)
controller = controller_nn.get_fn()

# Critic NN
critic_nn = nn.Critic_NN(mjx_model.nq*2 + mjx_model.nbiomtu*2 ,1)
critic_params, key = critic_nn.init_parameters(key)
criticer = critic_nn.get_fn()

# Test the two neural networks
print(controller(controller_params, jp.ones(mjx_model.nq*2+mjx_model.nbiomtu*2), key))
print(criticer(critic_params, jp.ones(mjx_model.nq*2+mjx_model.nbiomtu*2)))

(Array([0.209172, 0.215768], dtype=float32), Array([0.20899 , 0.215497], dtype=float32), Array([-0.49779 , -0.102347], dtype=float32))
[-1.48502]


In [4]:
from functools import partial
# Since the environment resets with init_mjx_data, the muscles are already in the equilibrum condition.
def reset(init_mjx_data, batch_size):
    new_data = jax.tree.map(
        partial(jp.repeat, repeats=batch_size, axis=0),
        jax.tree.map(partial(jp.expand_dims, axis=0),init_mjx_data))
    return new_data

def random_init(data, model, rng: jax.Array):
    nbiomtu = model.nq
    init_qpos = data.qpos
    init_qvel = data.qvel
    new_rng, rng1, rng2 = jax.random.split(rng, 3)
    # Qpos_1 is the vertical position
    random_qpos = init_qpos + jax.random.uniform(rng1, [nbiomtu], minval=jp.array(-1.0, dtype=jp.float32), maxval=jp.array(1.0, dtype=jp.float32))*0.01
    random_qvel = init_qvel + jax.random.uniform(rng2, [nbiomtu], minval=jp.array(-1.0, dtype=jp.float32), maxval=jp.array(1.0, dtype=jp.float32))*0.01
    newdata = data.replace(qpos=random_qpos)
    newdata = newdata.replace(qvel=random_qvel)
    newdata = mjx.forward(mjx_model, newdata)
    # print('data:',data.qpos, data.qvel)
    # Calculate equilibrum
    # newdata = acceleration_mtu.calc_equilibrium(mjx_model, newdata)
    # newdata = mjx_step(mjx_model, newdata)
    return newdata, new_rng

vrandom_init = jax.jit(jax.vmap(random_init, in_axes=(None, None, 0), out_axes=0))

In [5]:
import exp

def step_fn(carry, _):
    data, model = carry
    new_data = mjx.step(model, data)
    new_carry = (new_data, model)
    return new_carry, _

def multiple_steps(model, data):
    init_carry = (data, model)
    y, _ = jax.lax.scan(step_fn, init_carry, None, length=10)
    new_data = y[0]
    return new_data

def nn_mjx_one_step(nn_params, model, data, key):
    states = jp.concatenate([data.qpos, data.qvel, data.biomtu.fiber_l, data.biomtu.fiber_v])
    act, mean, std = controller(nn_params, states, key)
    # Generate the next key
    new_key = jax.random.split(key,1)[0]
    data = data.replace(biomtu = data.biomtu.replace(act = act))
    new_data = mjx.step(model, data)
    return new_data, new_key, act

def nn_mjx_perturbe_one_step(nn_params, model, data, key):
    states = jp.concatenate([data.qpos, data.qvel, data.biomtu.fiber_l, data.biomtu.fiber_v])
    act, mean, std = controller(nn_params, states, key)
    # Generate the next key
    new_key = jax.random.split(key,1)[0]
    data = data.replace(biomtu = data.biomtu.replace(act = act))
    xfrc = jax.random.normal(key,(mjx_model.nbody, 6))*1.0
    data = data.replace(xfrc_applied=xfrc)
    new_data = mjx.step(model, data)
    return new_data, new_key, act


@jax.jit
def jit_nn_mjx_one_step_no_random(nn_params, model, data, key):
    states = jp.concatenate([data.qpos, data.qvel, data.biomtu.fiber_l, data.biomtu.fiber_v])
    act, mean, std = controller(nn_params, states, key)
    # Generate the next key
    new_key = jax.random.split(key,1)[0]
    data = data.replace(biomtu = data.biomtu.replace(act = act))
    new_data = mjx.step(model, data)
    return new_data, new_key, mean

def nn_step_fn(carry, _):
    nn_params, model, data, key = carry
    new_data, new_key, act = nn_mjx_one_step(nn_params, model, data, key)
    # new_data, new_key, act = nn_mjx_perturbe_one_step(nn_params, model, data, key)
    new_carry = (nn_params, model, new_data, new_key)
    # Calculate reward
    states = jp.concat([data.qpos, data.qvel, data.biomtu.fiber_l, data.biomtu.fiber_v])
    next_states = jp.concat([new_data.qpos, new_data.qvel, new_data.biomtu.fiber_l, new_data.biomtu.fiber_v])
    action = act
    reward = -new_data.qpos[1]**2
    done = jp.abs(new_data.qpos[1])>0.8
    experience = exp.experience(states, next_states, action, reward, done)
    
    return new_carry, experience

def decay_sum_scan(x, decay):
    def f(sxtm1, xt):
        b = xt + decay * sxtm1
        return b, b
    return jax.lax.scan(f, jp.zeros(x.shape[1:]), x)[1]

@jax.jit
def jit_nn_multi_steps(nn_params, model, data, key):
    # Also deal with the done in the experience pool
    
    repeat_length = 32
    init_carry = (nn_params, model, data, key)
    y, experience = jax.lax.scan(nn_step_fn, init_carry, None, length=repeat_length)
    new_data = y[2]
    new_key = y[3]
    
    return new_data, new_key, experience

# @jax.jit
def v_nn_multi_steps(nn_params, model, data, keys):
    return jax.vmap(jit_nn_multi_steps, in_axes=(None, None, 0, 0))(nn_params, model, data, keys)

jit_v_nn_multi_steps = jax.jit(v_nn_multi_steps)

# This function generate
@jax.jit
def jit_vv_nn_multi_steps(nn_params, model, data, key):
    return jax.vmap(jit_v_nn_multi_steps, in_axes=(None, None, None, 1))(nn_params, model, data, key)

In [6]:
# Settings
import exp

critic_batch_size =500
controller_batch_size = 100
key = jax.random.key(2024)
keys = jax.random.split(key, controller_batch_size)

memory_settings = exp.memory_settings(critic_batch_size*5, mjx_model.nq*2+mjx_model.nbiomtu*2, mjx_model.nbiomtu, 1)

In [ ]:
exp_pool = None
datas = jax.jit(reset,static_argnames="batch_size")(init_mjx_data, controller_batch_size)
init_data_batch = datas
for i in range(10):
    datas, keys, exps = jit_v_nn_multi_steps(controller_params, mjx_model, datas, keys)
    # print(datas.qvel.shape, datas.ten_J.shape)
    exp_pool = exp.memory.add_exp(memory_settings, exp_pool, exps)
    
print(exp_pool.states.shape)

In [ ]:
import optax

v_criticer = jax.vmap(criticer,in_axes=(None, 0))
jit_v_criticer = jax.jit(v_criticer)

def critic_loss(params, batch):
    discount = 0.99
    states = batch.states
    next_states = batch.next_states
    actions = batch.actions
    rewards = batch.rewards
    
    critic_score = v_criticer(params, states)
    target = rewards + discount* jax.lax.stop_gradient(v_criticer(params, next_states))
    # target = rewards + discount* (v_criticer(params, next_states))
    
    loss = optax.l2_loss(critic_score, target)
    loss = jp.mean(loss)
    return loss

sample_batch = exp.memory.sample(exp_pool, critic_batch_size, key)
critic_loss_g_value_lower= jax.jit(jax.value_and_grad(critic_loss)).lower(critic_params, sample_batch)

jit_critic_loss_g_value = critic_loss_g_value_lower.compile()
a=jit_critic_loss_g_value.cost_analysis()[0]['flops']
print(a)

In [ ]:
def decay_sum_scan(x, decay):
    def f(sxtm1, xt):
        b = xt + decay * sxtm1
        return b, b
    return jax.lax.scan(f, jp.zeros(x.shape[1:]), x)[1]

def controller_loss_and_experience(controller_params, critic_params, batch, batch_size, mjx_model, init_data_batch, keys):
    # Generate data for simulation
    nq = mjx_model.nq
    nmtu = mjx_model.nbiomtu
    
    # Deal with the done state, reset the done state with init state
    # exp_data_batch = init_data_batch.replace(
    #     qpos = batch.states[:,0:nq], 
    #     qvel = batch.states[:,nq:nq*2],
    #     biomtu = init_data_batch.biomtu.replace(
    #         fiber_l = batch.states[nq*2, nq*2+nmtu],
    #         fiber_v = batch.states[nq*2+nmtu, nq*2+nmtu*2]
    #     ))
    
    qpos = jp.where(batch.dones==1, init_data_batch.qpos, batch.states[:,0:nq])
    qvel = jp.where(batch.dones==1, init_data_batch.qvel, batch.states[:,nq:nq*2])
    fiber_l = jp.where(batch.dones==1, init_data_batch.biomtu.fiber_l, batch.states[:,nq*2 : nq*2+nmtu])
    fiber_v = jp.where(batch.dones==1, init_data_batch.biomtu.fiber_v, batch.states[:,nq*2+nmtu : nq*2+nmtu*2])
    
    # in_data = batch.dones, init_data_batch, exp_data_batch)
    
    in_data = jax.lax.stop_gradient(init_data_batch.replace(
        qpos = qpos,
        qvel = qvel,
        biomtu = init_data_batch.biomtu.replace(
            fiber_l = fiber_l,
            fiber_v = fiber_v)
        )
    )
    
    out_data, keys, new_exps = v_nn_multi_steps(controller_params, mjx_model, in_data, keys)
    # out_states = new_exps.next_states
    # jax.debug.print("out_states shape{0}", out_states.shape)
    out_states = jp.concat([out_data.qpos, out_data.qvel, out_data.biomtu.fiber_l, out_data.biomtu.fiber_v, out_data.sensordata],axis=1)
    critic_score = jp.squeeze(v_criticer(critic_params, out_states))
    decay_M = 0.99 ** jp.arange(new_exps.rewards.shape[1]-1)
    # print(new_exps.rewards.shape)
    loss = -jp.mean(decay_M@new_exps.rewards.T[:-1,:])
    # loss = -jp.mean(decay_M@new_exps.rewards.T[:-1,:]) - jp.mean(critic_score)
    # loss = -jp.mean(critic_score)
    return loss, new_exps

# The function calculating the loss of the controller and also generate experiences
g_loss_experience = jax.value_and_grad(controller_loss_and_experience, has_aux=True)

controller_keys = jax.random.split(key, controller_batch_size)
sample_batch = exp.memory.sample(exp_pool, controller_batch_size, key)

print("lowering")
g_loss_experience_lower = jax.jit(g_loss_experience, static_argnames=["batch_size"]).lower(
    controller_params, 
    critic_params, 
    sample_batch, 
    controller_batch_size, 
    mjx_model, 
    init_data_batch, 
    controller_keys)

print("compiling")
jit_g_loss_experience = g_loss_experience_lower.compile()

b = jit_g_loss_experience.cost_analysis()[0]['flops']
print(b)

In [ ]:
print(sample_batch.states.shape)

b/a

## Train the two Neural Networks

In [12]:
import optax
key = jax.random.key(3454365435)

critic_params, key = critic_nn.init_parameters(key)
critic_tx = optax.apply_if_finite(optax.adam(learning_rate=1e-6), max_consecutive_errors=10)
# critic_tx = optax.apply_if_finite(optax.sgd(learning_rate=1e-5), max_consecutive_errors=50)
critic_opt_state = critic_tx.init(critic_params)
jit_critic_tx_update = jax.jit(critic_tx.update)


controller_params, key = controller_nn.init_parameters(key)
controller_tx = optax.apply_if_finite(optax.adam(learning_rate=1e-5), max_consecutive_errors=50)
# controller_tx = optax.apply_if_finite(optax.sgd(learning_rate=1e-2), max_consecutive_errors=50)
# controller_tx = optax.sgd(learning_rate=1e-6)
controller_opt_state = controller_tx.init(controller_params)
jit_controller_tx_update = jax.jit(controller_tx.update)

jit_apply_update = jax.jit(optax.apply_updates)


jit_sample = jax.jit(exp.memory.sample, static_argnames="batch_size")
jit_add_exp = jax.jit(exp.memory.add_exp, static_argnames="settings")

print(controller_params)

{'params': {'linear1': {'kernel': Array([[-0.521868, -0.223058, -0.413956, ...,  0.708522,  0.49591 ,  0.465597],
       [-0.092645,  0.019458, -0.3183  , ..., -0.489226, -0.267101,  0.305509],
       [ 0.203467,  0.342604, -0.404305, ...,  0.114232,  0.61902 ,  0.532535],
       ...,
       [-0.289248,  0.179362, -0.328398, ...,  0.45507 , -0.071455, -0.149657],
       [-0.321705,  0.214219,  0.25602 , ...,  0.120517, -0.50708 ,  0.535591],
       [-0.527762, -0.095684, -0.075469, ..., -0.669847, -0.029087,  0.397224]], dtype=float32), 'bias': Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

## Init the training data


In [7]:
def init_data(key,mjx_model):
    mjx_data = mjx.make_data(mjx_model)
    keys = key = jax.random.split(key,4)
    random_qpos = jax.random.uniform(keys[0], [2], minval=jp.array(-1.0, dtype=jp.float32), maxval=jp.array(1.0, dtype=jp.float32))
    random_qvel = jax.random.uniform(keys[1], [2], minval=jp.array(-1.0, dtype=jp.float32), maxval=jp.array(1.0, dtype=jp.float32))*0.1
    
    random_qpos = random_qpos.at[0].set(random_qpos[0]*0.1)
    random_qpos = random_qpos.at[1].set(random_qpos[1]*0.3)
    
    mjx_data = mjx_data.replace(qpos = random_qpos, qvel = random_qvel)

    # Calculate equilibrum
    mjx_data = acceleration_mtu.calc_equilibrium(mjx_model, mjx_data)
    
    return mjx_data

jit_v_init_data = jax.jit(jax.vmap(init_data, in_axes=(0,None)))




In [8]:
# Test 
key = jax.random.key(4532)
keys = jax.random.split(key, controller_batch_size)

print(keys.shape)

test_data = jit_v_init_data(keys, mjx_model)
print(test_data.qpos)
print("___________")
print(test_data.qvel)

(100,)
[[-0.047606  0.245926]
 [ 0.018174 -0.010012]
 [ 0.038285 -0.25566 ]
 [ 0.092525 -0.181915]
 [-0.03747   0.190355]
 [-0.07585  -0.165863]
 [ 0.0894   -0.024966]
 [ 0.08695   0.048732]
 [ 0.095481  0.293775]
 [-0.011247  0.266156]
 [-0.032044  0.207545]
 [-0.068613  0.036631]
 [ 0.088801 -0.069611]
 [-0.015412 -0.071564]
 [ 0.077661  0.111434]
 [ 0.050826  0.062921]
 [ 0.081484 -0.281168]
 [-0.014611  0.201475]
 [-0.092305 -0.251995]
 [ 0.058024  0.145313]
 [ 0.01638   0.071555]
 [ 0.058805 -0.079044]
 [-0.035041 -0.266214]
 [-0.083401  0.121915]
 [ 0.043826 -0.028575]
 [-0.094484  0.001002]
 [-0.022824  0.298112]
 [ 0.032444  0.229646]
 [-0.051935  0.127979]
 [-0.046064 -0.223181]
 [ 0.017213  0.165302]
 [ 0.074713 -0.06714 ]
 [-0.070857 -0.133987]
 [ 0.019137  0.071649]
 [ 0.049287 -0.042739]
 [-0.003312 -0.189575]
 [-0.038898 -0.136091]
 [ 0.087671  0.187513]
 [-0.064839  0.142492]
 [-0.070903  0.036709]
 [-0.005474 -0.154675]
 [-0.050284 -0.03171 ]
 [-0.098367 -0.291935]
 [ 0

In [10]:
def decay_sum_scan(x, decay):
    def f(sxtm1, xt):
        b = xt + decay * sxtm1
        return b, b
    return jax.lax.scan(f, jp.zeros(x.shape[1:]), x)[1]

def controller_loss(controller_params, critic_params, batch, batch_size, mjx_model, keys):
    # Generate data for simulation
    nq = mjx_model.nq
    nmtu = mjx_model.nbiomtu
    
    in_data = batch
    
    out_data, keys, new_exps = v_nn_multi_steps(controller_params, mjx_model, in_data, keys)
    # out_states = new_exps.next_states
    # jax.debug.print("out_states shape{0}", out_states.shape)
    out_states = jp.concat([out_data.qpos, out_data.qvel, out_data.biomtu.fiber_l, out_data.biomtu.fiber_v, out_data.sensordata],axis=1)
    decay_M = 0.99 ** jp.arange(new_exps.rewards.shape[1]-1)
    # print(new_exps.rewards.shape)
    loss = -jp.mean(decay_M@new_exps.rewards.T[:-1,:])
    # loss = -jp.mean(decay_M@new_exps.rewards.T[:-1,:]) - jp.mean(critic_score)
    # loss = -jp.mean(critic_score)
    return loss

# The function calculating the loss of the controller and also generate experiences
g_loss = jax.value_and_grad(controller_loss, has_aux=True)

controller_keys = jax.random.split(key, controller_batch_size)

print("lowering")
g_loss_lower = jax.jit(g_loss, static_argnames=["batch_size"]).lower(
    controller_params, 
    critic_params, 
    test_data, 
    controller_batch_size, 
    mjx_model, 
    controller_keys)

print("compiling")
jit_g_loss = g_loss_lower.compile()

b = jit_g_loss.cost_analysis()[0]['flops']
print(b)

lowering


TypeError: expected function with aux output to return a two-element tuple, but got type <class 'jax._src.interpreters.ad.JVPTracer'> with value Traced<ShapedArray(float32[])>with<JVPTrace(level=3/0)> with
  primal = Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=2/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7e2c4bf92bd0>, in_tracers=(Traced<ShapedArray(float32[]):JaxprTrace(level=2/0)>,), out_tracer_refs=[<weakref at 0x7e2d1028e8e0; to 'JaxprTracer' at 0x7e2d1028e610>], out_avals=[ShapedArray(float32[])], primitive=pjit, params={'jaxpr': { lambda ; a:f32[]. let b:f32[] = neg a in (b,) }, 'in_shardings': (UnspecifiedValue,), 'out_shardings': (UnspecifiedValue,), 'in_layouts': (None,), 'out_layouts': (None,), 'resource_env': None, 'donated_invars': (False,), 'name': '_negative', 'keep_unused': False, 'inline': True}, effects=set(), source_info=<

In [ ]:
key = jax.random.key(2024)

for i in range(500):
    key = jax.random.split(key, 1)[0]
    keys = jax.random.split(key, controller_batch_size)
    
    train_data = jit_v_init_data(keys, mjx_model)
    
    print("-----------",i,"-------------")
    
    for i in range(10):
        controller_loss_exps, controller_loss_grad = jit_g_loss(
                controller_params, critic_params, train_data, mjx_model, keys)
        
        controller_loss = controller_loss_exps[0]
        exps = controller_loss_exps[1]
        
        # Update params
        controller_updates, controller_opt_state = jit_controller_tx_update(controller_loss_grad, controller_opt_state)
        # print(controller_updates)
        controller_params = jit_apply_update(controller_params, controller_updates)
        
        print("Controller Loss:", controller_loss)
    

----------- 0 -------------


TypeError: Function compiled with input pytree does not match the input pytree it was called with. There are 1 mismatches, including:
    * at args[2], seen <class 'exp.experience'> but now given <class 'mujoco.mjx._src.types.Data'>, so their Python types differ

In [ ]:
jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_debug_nans", False)
import exp


death_rate_buf = []
best_death_rate = 0.5

for i in range(4000):

    print("----------",i,"----------")
    # criric_repeat = 1
    # if(i >= 200):
    #     criric_repeat = 1
        
    # for j in range(criric_repeat):
    #     batch = jit_sample(exp_pool, critic_batch_size, key)
    #     key = jax.random.split(key, 1)[0]
        
    #     # criticer
    #     critic_loss, critic_loss_grad = jit_critic_loss_g_value(critic_params, batch)
        
    #     # Update params
    #     critic_updates, critic_opt_state = jit_critic_tx_update(critic_loss_grad, critic_opt_state)
    #     critic_params = jit_apply_update(critic_params, critic_updates)
    
    # print("criticer loss:",critic_loss)
    # controller
    
    controller_repeat = 1
    # if(i >= 10):
        # controller_repeat = 5
    
    for j in range(controller_repeat):
        keys = jax.random.split(key,controller_batch_size)
        controller_batch = jit_sample(exp_pool, controller_batch_size, key)
        key = jax.random.split(key, 1)[0]
        controller_loss_exps, controller_loss_grad = jit_g_loss_experience(
            controller_params, critic_params, controller_batch, mjx_model, init_data_batch, keys)
        controller_loss = controller_loss_exps[0]
        exps = controller_loss_exps[1]
            
        # Update params
        controller_updates, controller_opt_state = jit_controller_tx_update(controller_loss_grad, controller_opt_state)
        # print(controller_updates)
        controller_params = jit_apply_update(controller_params, controller_updates)
    
    # exp_pool = jit_add_exp(memory_settings, exp_pool, exps)
    
    # Count Dones
    death_rate_buf.insert(0, jp.count_nonzero( exps.dones)/(exps.dones.shape[1]*controller_batch_size))
    death_rate_buf = death_rate_buf[:50]
    death_rate = np.mean(death_rate_buf)
    
    print("Controller Loss:", controller_loss)
    mean_reward = jp.mean(exps.rewards)
    print("mean rewards:", mean_reward)
    mean_score = jp.mean(jit_v_criticer(critic_params, exps.states))
    print("mean score:", mean_score)
    print("scorc/reward:", mean_score/mean_reward)
    print("death rate:", death_rate)
    # print(controller_loss_grad)
    
    if(best_death_rate > death_rate): best_death_rate = death_rate
    # print("best_death_rate:", best_death_rate)
    # add exps
    
    # print(controller_loss_grad)
    print(jax.tree.map(jp.max,controller_loss_grad))
    # print(controller_params)
    print(controller_updates['params']['linear1']['kernel'])
    print(controller_updates['params']['linear2']['kernel'])
    print(controller_updates['params']['linear3']['kernel'])
    print(controller_updates['params']['linear4']['kernel'])
    print(controller_updates['params']['linear5']['kernel'])
    if(i%20 == 0):
        plt.figure()
        plt.plot(exp_pool.rewards)
        plt.plot()
        # plt.plot(exp_pool.rewards)

In [ ]:
# print(controller_params['params']['linear1']['kernel'])
# print(controller_params['params']['linear2']['kernel'])
# print(controller_params['params']['linear3']['kernel'])
# print(controller_params['params']['linear4']['kernel'])
# print(controller_params['params']['linear5']['kernel'])
print(controller_params)

In [ ]:
import mujoco.viewer
import time


mjx_data = mjx.make_data(mjx_model)
mj_data = mujoco.MjData(mj_model)
previous_frame_time = time.time()
i = 0
key = jax.random.key(334)
with mujoco.viewer.launch_passive(mj_model, mj_data) as viewer:
    while viewer.is_running():
        # Update mjx_data from mj_data. The mj_data was modified by the viewer
        # mjx_data = mjx_data.replace(ctrl=mj_data.ctrl, xfrc_applied=mj_data.xfrc_applied)
        # Use the nerual network to generate ctrl signal
        # Generate key
        mjx_data = mjx_data.replace(xfrc_applied=jp.array(mj_data.xfrc_applied, dtype=jp.float32))
        mjx_data = mjx_data.replace(
            qpos= jp.array(mj_data.qpos, dtype=jp.float32),
            qvel= jp.array(mj_data.qvel, dtype=jp.float32),
            time = jp.array(mj_data.time, dtype=jp.float32))
        
        # Update mjx_model from mj_model
        mjx_model = mjx_model.tree_replace({
            'opt.gravity': jp.array(mj_model.opt.gravity, dtype=jp.float32),
            'opt.tolerance': jp.array(mj_model.opt.tolerance, dtype=jp.float32),
            'opt.ls_tolerance': jp.array(mj_model.opt.ls_tolerance, dtype=jp.float32),
            'opt.timestep': jp.array(mj_model.opt.timestep, dtype=jp.float32),
        })
        
        key = jax.random.split(key,1)[0]
        # mjx_data = mjx_step(mjx_model, mjx_data)
        # mjx_data, loss, expp = jit_nn_multi_steps(controller_params, mjx_model, mjx_data, key)
        mjx_data, key, act = jit_nn_mjx_one_step_no_random(controller_params, mjx_model, mjx_data, key)
        mjx.get_data_into(mj_data, mj_model, mjx_data)
        
        # Record the current time at the start of this frame
        current_frame_time = time.time()
    
        # Calculate the difference in time from the last frame
        time_between_frames = current_frame_time - previous_frame_time
    
        # Print the time between frames
        print(f"Time between frames: {time_between_frames} seconds")
        previous_frame_time = current_frame_time
        
        print("ACT:", mjx_data.biomtu.act)
        print(mjx_data.qpos)
        # print(mjx_data.sensordata)
        # print(len(mjx_data.qvel))
        viewer.sync()

In [ ]:
import jax.numpy as jp
0.99 ** jp.arange(10)

In [ ]:
import jax
import jax.numpy as jp

def test_fn(x,carry):
    x0 = carry
    carry = x
    return x0+x, carry

xs = jp.linspace(0,10,11)

x, carry = jax.lax.scan(test_fn, 0, xs)

print(x,carry)